In [1]:
#Mindex Coding Assignment

In [2]:
from etl_functions import *
import pandas as pd
from sqlalchemy import create_engine

In [3]:
#Use helper function to download all files from S3 and save locally
download_raw_files()

starting file downloads
downloaded all files from s3


In [4]:
#Read in csv files to pandas dataframes
bengals = pd.read_csv("raw_data/bengals.csv").set_index('Week')
boyd = pd.read_csv("raw_data/boyd_receiving.csv").set_index('Week')
chase = pd.read_csv("raw_data/chase_receiving.csv").set_index('Week')
higgins = pd.read_csv("raw_data/higgins_receiving.csv").set_index('Week')

In [5]:
#join dfs together on 'week' field
joined = bengals.join(boyd, rsuffix='_boyd').join(chase, rsuffix='_chase').join(higgins, rsuffix='_higgins') \
         .rename(columns={'Yards': 'Yards_boyd', 'TD': 'TD_boyd'}) \
         .reset_index()
joined.columns = map(str.lower, joined.columns) #lowercase column names

In [6]:
#Display wins/losses as text
joined['result'] = joined['result'].apply(map_wins_losses)

In [7]:
#Write to Postgresql DB
engine = create_engine(CONNECTION_STRING)
joined.to_sql('ray_sidener', engine, if_exists='replace')

25